In [1]:
import os
os.environ["THEANO_FLAGS"] = "floatX=float32,device=cpu"

import deepthought
import os
from deepthought.datasets.eeg.meta_class import DataFileWithMetaClasses

db = DataFileWithMetaClasses(
        filepath='data/OpenMIIR-Perception-512Hz.pklz',
        meta_classes={                                                
                    'within_subject_tuples_group': ['subject', 'condition'],
                    'cross_subject_tuples_group': ['condition'], # cross-participant triplets - this should be very tough                },
                },
    )

In [57]:
from deepthought.datasets.eeg.EEGEpochsDataset import EEGEpochsDataset
INPUT_LEN = 6000
base_dataset = EEGEpochsDataset(
                db=db,
                meta_sources=['subject'],
                selectors={         
                    #'trial_no' : [0,1,  3,4],
                    'condition' : [1,2,3,4],
                },
                use_targets=False,
                stop_sample=INPUT_LEN,
                label_attribute='stimulus_id',
                label_map= { # stimulus_id
                    1: 0, 2: 1, 3: 2, 4: 3, 11: 4, 12: 5, 13: 6, 14: 7, 21: 8, 22: 9, 23: 10, 24: 11,
                },
                remove_dc_offset=True,
                layout='tf',
            )

In [58]:
print (base_dataset.trials.shape)
print (base_dataset.y.shape)
print (base_dataset.metadata[0])
print (base_dataset.metadata[0].keys())

(2160, 6000, 1, 64)
(2160, 12)
{'subject': 'P01', 'trial_type': 'perception', 'trial_no': 0, 'condition': 1, 'stimulus_id': 1, 'within_subject_tuples_group': 0, 'cross_subject_tuples_group': 0}
dict_keys(['subject', 'trial_type', 'trial_no', 'condition', 'stimulus_id', 'within_subject_tuples_group', 'cross_subject_tuples_group'])


In [59]:
# convert to bc01 axis format
import numpy as np
features_bc01 = np.rollaxis(base_dataset.trials,3,1)
print (features_bc01.shape)

(2160, 64, 6000, 1)


In [388]:
# save dataset

import h5py
from fuel.datasets.hdf5 import H5PYDataset

hdf5name = 'data/OpenMIIR-Perception-512Hz.hdf5'
f = h5py.File(hdf5name, mode='w')

N = len(base_dataset.trials)
fx = f.create_dataset('features', features_bc01.shape, dtype='float32')
fy = f.create_dataset('targets', base_dataset.y.shape, dtype='int8')
fx[...] = features_bc01
fy[...] = base_dataset.y

# for using indices
findices = f.create_dataset('indices', (N,), dtype='int')
findices[...] = range(N)

# additional metadata source for hydra-net control flow
subjects = ['P01','P04','P06','P07','P09','P11','P12','P13','P14']
fsubject = f.create_dataset('subjects', (N,), dtype='int8')
fsubject[...] = [subjects.index(meta['subject']) for meta in base_dataset.metadata]

split_dict = {
    'all': {'features': (0, N), 
            'indices': (0, N), 
            'targets': (0, N),
            'subjects': (0, N)
           },
}

f.attrs['split'] = H5PYDataset.create_split_array(split_dict)

f.flush()
f.close()

# metadata for subset selection
import deepthought.util.fs_util as fs_util
fs_util.save(hdf5name+'.meta.pklz', base_dataset.metadata)

C:\Users\varun\Documents\Deep Learning\project\openmiir-rl-2016-master\openmiir-rl-2016-master\fuel\datasets\hdf5.py:270: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('comment', 'a', comment_len)]))


In [60]:
features_bc01.shape

(2160, 64, 6000, 1)

In [61]:
base_dataset.y.shape

(2160, 12)

In [62]:
base_dataset.y[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [63]:
base_dataset.y[60]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [64]:
labels=[]
for i in base_dataset.y:
    labels.append(np.where(i==1))
labels=np.array(labels)
labels=labels.reshape((base_dataset.y.shape[0],))
labels.shape

(2160,)

In [65]:
labels[5]

0

In [66]:
labels_dict={0:1,1:2,2:3,3:4,4:11,5:12,6:13,7:14,8:21,9:22,10:23,11:24}

In [67]:
labels

array([ 0,  0,  0, ..., 11, 11, 11], dtype=int64)

In [68]:
features=features_bc01

In [69]:
features.shape

(2160, 64, 6000, 1)

In [70]:
features=features.reshape(features.shape[0],64,INPUT_LEN)
#features=np.swapaxes(features,1,2)
features.shape

(2160, 64, 6000)

In [71]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional,SimpleRNN, Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku

In [72]:
x_train=features[:1680]
y_train=labels[:1680]
x_test=features[1680:1920]
y_test=labels[1680:1920]
x_val=features[1920:]
y_val=labels[1920:]

print(x_train.shape,y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

(1680, 64, 6000) (1680,) (240, 64, 6000) (240,) (240, 64, 6000) (240,)


Execute the below cell for within-subject evaluation

In [100]:
#x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size = 0.3, random_state = 5)
x_val=features[4::5]
y_val=labels[4::5]
x_train=np.delete(features, list(range(0, features.shape[0], 5)), axis=0)
y_train=np.delete(labels, list(range(0, labels.shape[0], 5)), axis=0)

x_test=x_train[3::4]
y_test=y_train[3::4]
x_train=np.delete(x_train, list(range(0, features.shape[0], 4)), axis=0)
y_train=np.delete(y_train, list(range(0, labels.shape[0], 4)), axis=0)


print(x_train.shape,y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

C:\Users\varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  if __name__ == '__main__':


(1296, 64, 3518) (1296,) (432, 64, 3518) (432,) (432, 64, 3518) (432,)


C:\Users\varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  # Remove the CWD from sys.path while we load stuff.


In [73]:
#x_train=x_train.to_numpy()
x_train = x_train.reshape(x_train.shape[0], 64,INPUT_LEN)
#x_val=x_val.to_numpy()
x_val = x_val.reshape(x_val.shape[0],  64,INPUT_LEN)


In [75]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences = True),input_shape=(64,INPUT_LEN)))
model.add(Dropout(0.7))
model.add(Bidirectional(LSTM(64)))
#model.add(LSTM(100))
model.add(Dropout(0.7))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [77]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_8 (Bidirection (None, 64, 128)           3105280   
_________________________________________________________________
dropout_8 (Dropout)          (None, 64, 128)           0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 12)               

In [78]:
history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=25, batch_size=32)

Train on 1680 samples, validate on 240 samples
Epoch 1/25
1680/1680 [==============================] - 57s 34ms/sample - loss: 2.5705 - accuracy: 0.0851 - val_loss: 2.4712 - val_accuracy: 0.1125
Epoch 2/25
1680/1680 [==============================] - 48s 28ms/sample - loss: 2.3524 - accuracy: 0.1905 - val_loss: 2.3910 - val_accuracy: 0.1125
Epoch 3/25
1680/1680 [==============================] - 42s 25ms/sample - loss: 2.0618 - accuracy: 0.3089 - val_loss: 2.2435 - val_accuracy: 0.1625
Epoch 4/25
1680/1680 [==============================] - 36s 21ms/sample - loss: 1.6641 - accuracy: 0.4298 - val_loss: 2.1539 - val_accuracy: 0.1875
Epoch 5/25
1680/1680 [==============================] - 36s 21ms/sample - loss: 1.3215 - accuracy: 0.5506 - val_loss: 2.3262 - val_accuracy: 0.1958
Epoch 6/25
1680/1680 [==============================] - 36s 21ms/sample - loss: 0.9558 - accuracy: 0.6792 - val_loss: 2.6685 - val_accuracy: 0.2042
Epoch 7/25
1680/1680 [==============================] - 36s 22ms/

In [79]:
yhat=model.predict(x_test)
yhat=yhat.round()
yhat=np.argmax(yhat,axis = 1)
print('Accuracy:',np.sum(yhat==y_test)/y_test.size*100)

Accuracy: 23.333333333333332


In [80]:
cm = np.zeros((12,12), dtype=int)
np.add.at(cm, [y_test, yhat], 1)
cm

C:\Users\varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


array([[3, 0, 0, 3, 6, 0, 0, 3, 0, 5, 0, 0],
       [0, 2, 3, 0, 0, 6, 2, 0, 5, 1, 0, 1],
       [3, 2, 4, 2, 0, 0, 2, 1, 1, 3, 2, 0],
       [5, 0, 1, 1, 8, 0, 0, 2, 0, 2, 1, 0],
       [3, 0, 2, 4, 5, 0, 2, 3, 0, 0, 1, 0],
       [2, 2, 1, 0, 0, 7, 1, 0, 3, 0, 2, 2],
       [1, 3, 5, 0, 1, 0, 6, 0, 2, 0, 1, 1],
       [2, 0, 1, 5, 3, 1, 1, 6, 0, 1, 0, 0],
       [1, 2, 1, 1, 0, 5, 2, 0, 7, 0, 0, 1],
       [5, 0, 0, 3, 7, 0, 0, 1, 0, 2, 1, 1],
       [0, 1, 3, 1, 1, 1, 2, 0, 4, 0, 4, 3],
       [0, 5, 0, 0, 0, 5, 0, 0, 1, 0, 0, 9]])